In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'2.0.20'

In [3]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.sql import select, insert, update, delete, join, func

In [64]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [65]:
# cur.excute/many/script
con = engine.connect()

In [7]:
con.execute('SELECT 1')

ObjectNotExecutableError: Not an executable object: 'SELECT 1'

In [8]:
meta = MetaData()

In [9]:
meta.tables

FacadeDict({})

In [53]:
Table('USER', meta,
     Column('PK', Integer, primary_key=True),
      Column('NAME', Text, nullable=False))

Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)

In [11]:
len(meta.tables)

1

In [15]:
User = meta.tables['USER']

In [16]:
print(User.insert())

INSERT INTO "USER" ("PK", "NAME") VALUES (:PK, :NAME)


In [17]:
meta.create_all(engine) #engine보고 일해라고 하는거임. 

2023-09-06 09:24:18,550 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 09:24:18,552 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-06 09:24:18,552 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:24:18,552 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-06 09:24:18,554 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:24:18,555 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT NOT NULL, 
	PRIMARY KEY ("PK")
)


2023-09-06 09:24:18,555 INFO sqlalchemy.engine.Engine [no key 0.00038s] ()
2023-09-06 09:24:18,555 INFO sqlalchemy.engine.Engine COMMIT


In [66]:
meta.drop_all(engine) #다 날려볼거임

2023-09-06 09:51:27,636 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 09:51:27,636 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-06 09:51:27,641 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:27,642 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-06 09:51:27,642 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:27,643 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TEMP")
2023-09-06 09:51:27,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:27,644 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TEMP")
2023-09-06 09:51:27,644 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:27,645 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-06 09:51:27,646 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:27,647 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-06 09:51:27,647 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 

In [19]:
#바인딩을 시켰기 때문에 엔진을 통해서 데이터베이스에 있는것도 날라갔다.

In [67]:
meta.tables

FacadeDict({'ADDRESS': Table('ADDRESS', MetaData(), Column('PK', Integer(), table=<ADDRESS>, primary_key=True, nullable=False), Column('NAME', Text(), table=<ADDRESS>), Column('FK', Integer(), ForeignKey('USER.PK'), table=<ADDRESS>), schema=None), 'TEMP': Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>, primary_key=True, nullable=False), Column('NAME', Text(), table=<TEMP>), Column('FK', Integer(), table=<TEMP>, nullable=False), schema=None), 'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)})

In [21]:
#어라.. 하지만 다시 살아있다. 원래 살아있으면 안된다고 한다.

In [68]:
meta.remove(User) #DB에는 테이블이 남아있고, 객체로써의 테이블은 살아진 상태가 된다. 싱크가 안맞는 상황이 오는거다.

In [69]:
meta.tables

FacadeDict({'ADDRESS': Table('ADDRESS', MetaData(), Column('PK', Integer(), table=<ADDRESS>, primary_key=True, nullable=False), Column('NAME', Text(), table=<ADDRESS>), Column('FK', Integer(), ForeignKey('USER.PK'), table=<ADDRESS>), schema=None), 'TEMP': Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>, primary_key=True, nullable=False), Column('NAME', Text(), table=<TEMP>), Column('FK', Integer(), table=<TEMP>, nullable=False), schema=None)})

In [70]:
meta.reflect(engine)

2023-09-06 09:51:42,584 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 09:51:42,584 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-09-06 09:51:42,597 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:42,598 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-09-06 09:51:42,598 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:51:42,599 INFO sqlalchemy.engine.Engine ROLLBACK


In [71]:
meta.tables

FacadeDict({'ADDRESS': Table('ADDRESS', MetaData(), Column('PK', Integer(), table=<ADDRESS>, primary_key=True, nullable=False), Column('NAME', Text(), table=<ADDRESS>), Column('FK', Integer(), ForeignKey('USER.PK'), table=<ADDRESS>), schema=None), 'TEMP': Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>, primary_key=True, nullable=False), Column('NAME', Text(), table=<TEMP>), Column('FK', Integer(), table=<TEMP>, nullable=False), schema=None)})

In [29]:
#다시 반영을 시켜주는 명령을 썼는데 난 왜 안뜨지.....

In [72]:
dir(User.select()) #여기 있는애들이 다 sql문으로 바뀔 수 있는 애들이다. 명령문들

['__annotations__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_add_context_option',
 '_all_selected_columns',
 '_annotate',
 '_annotations',
 '_annotations_cache_key',
 '_assert_no_memoizations',
 '_attributes',
 '_auto_correlate',
 '_cache_key_traversal',
 '_clone',
 '_clone_annotations_traverse_internals',
 '_cloned_set',
 '_compile_options',
 '_compile_state_factory',
 '_compile_state_plugin',
 '_compile_w_cache',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_correlate',
 '_correlate_except',


In [73]:
print(User.insert().values(PK=1, NAME='아무개'))

INSERT INTO "USER" ("PK", "NAME") VALUES (:PK, :NAME)


In [74]:
print(User.insert())

INSERT INTO "USER" ("PK", "NAME") VALUES (:PK, :NAME)


In [75]:
print(User.insert().values(PK=1, NAME='아무개').compile().params)

{'PK': 1, 'NAME': '아무개'}


In [35]:
#아직까지는 객체에서 일어나는 일이라서 DB랑은 아무상관이 없다.

In [76]:
con.execute(User.insert().values(PK=1, NAME='아무개'))

2023-09-06 09:51:57,673 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 09:51:57,675 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("PK", "NAME") VALUES (?, ?)
2023-09-06 09:51:57,675 INFO sqlalchemy.engine.Engine [generated in 0.00260s] (1, '아무개')


OperationalError: (sqlite3.OperationalError) no such table: USER
[SQL: INSERT INTO "USER" ("PK", "NAME") VALUES (?, ?)]
[parameters: (1, '아무개')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [63]:
con.execute(User.select()).fetchall()

2023-09-06 09:43:39,318 INFO sqlalchemy.engine.Engine SELECT "USER"."PK", "USER"."NAME" 
FROM "USER"
2023-09-06 09:43:39,318 INFO sqlalchemy.engine.Engine [cached since 725.6s ago] ()


OperationalError: (sqlite3.OperationalError) no such table: USER
[SQL: SELECT "USER"."PK", "USER"."NAME" 
FROM "USER"]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [38]:
#난 왜 에러 파티냐~~~~~^^

In [40]:
type(con.execute(User.select()))

2023-09-06 09:34:19,214 INFO sqlalchemy.engine.Engine SELECT "USER"."PK", "USER"."NAME" 
FROM "USER"
2023-09-06 09:34:19,217 INFO sqlalchemy.engine.Engine [cached since 165.4s ago] ()


OperationalError: (sqlite3.OperationalError) no such table: USER
[SQL: SELECT "USER"."PK", "USER"."NAME" 
FROM "USER"]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [41]:
#join 까지 해보게 테이블 하나 더 만들겠다

In [54]:
Table('ADDRESS',meta,
     Column('PK', Integer, primary_key=True),
      Column('NAME', Text),
      Column('FK', Integer, ForeignKey('USER.PK')))

InvalidRequestError: Table 'ADDRESS' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [43]:
User.columns.keys()

['PK', 'NAME']

In [44]:
User.columns['PK']


Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False)

In [45]:
meta.create_all(engine)

2023-09-06 09:37:34,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 09:37:34,235 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-06 09:37:34,235 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:37:34,235 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-06 09:37:34,235 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:37:34,235 INFO sqlalchemy.engine.Engine ROLLBACK


NoReferencedTableError: Foreign key associated with column 'ADDRESS.FK' could not find table 'USER' with which to generate a foreign key to target column 'PK'

In [46]:
#왜 또 에러고..

In [47]:
Table('TEMP',meta,
     Column('PK', Integer, primary_key=True),
      Column('NAME', Text),
      Column('FK', Integer, nullable=False))

Table('TEMP', MetaData(), Column('PK', Integer(), table=<TEMP>, primary_key=True, nullable=False), Column('NAME', Text(), table=<TEMP>), Column('FK', Integer(), table=<TEMP>, nullable=False), schema=None)

In [48]:
User.columns['PK']

Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False)

In [49]:
meta.create_all(engine)

2023-09-06 09:39:22,843 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-06 09:39:22,844 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-06 09:39:22,844 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:39:22,846 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-06 09:39:22,846 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:39:22,847 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TEMP")
2023-09-06 09:39:22,847 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:39:22,848 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TEMP")
2023-09-06 09:39:22,848 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-06 09:39:22,849 INFO sqlalchemy.engine.Engine ROLLBACK


NoReferencedTableError: Foreign key associated with column 'ADDRESS.FK' could not find table 'USER' with which to generate a foreign key to target column 'PK'

In [51]:
con.execute(
    meta.tables['ADDRESS'].insert().values(PK=1, NAME='주소', FK=1)
)

2023-09-06 09:40:40,191 INFO sqlalchemy.engine.Engine INSERT INTO "ADDRESS" ("PK", "NAME", "FK") VALUES (?, ?, ?)
2023-09-06 09:40:40,191 INFO sqlalchemy.engine.Engine [generated in 0.00175s] (1, '주소', 1)


OperationalError: (sqlite3.OperationalError) no such table: ADDRESS
[SQL: INSERT INTO "ADDRESS" ("PK", "NAME", "FK") VALUES (?, ?, ?)]
[parameters: (1, '주소', 1)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [52]:
con.execute(
    meta.tables['TEMP'].insert().values(PK=1, NAME='가짜', FK=1)
)

2023-09-06 09:41:01,471 INFO sqlalchemy.engine.Engine INSERT INTO "TEMP" ("PK", "NAME", "FK") VALUES (?, ?, ?)
2023-09-06 09:41:01,472 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (1, '가짜', 1)


OperationalError: (sqlite3.OperationalError) no such table: TEMP
[SQL: INSERT INTO "TEMP" ("PK", "NAME", "FK") VALUES (?, ?, ?)]
[parameters: (1, '가짜', 1)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [55]:
#이 둘의 차이는 FK를 줬냐 안줬냐의 차이이다. 

In [ ]:
#다른 파일에서 DROP 부분을 안쓰고 다시함.